In [14]:
import csv
import json
import pandas as pd
from datetime import datetime
import numpy as np

from collections import Counter

outputFolders = "../data_2019_processed/"

input_file_sufix = "_filtred.csv"
fileOutPutSufixName = "_transformed.csv"

outputFoldersWeek = outputFolders + "w"

numberOfWeeks = 4 # each week will have the WeekTime, but the rest not

In [15]:
def convert_to_decibels(value):
    if(value > 0.0):
        return 20 * (np.log(32768) - np.log(value/32768) )
        #return 20 * ( math.log(32768/value) )
        #return 20 * (np.log(value)) 
    else:
        return 0.0

def treat_data_to_0_1(value):
    if(value > 0.0):
        return 1.0
        #return 20 * ( math.log(32768/value) )
        #return 20 * (np.log(value)) 
    else:
        return 0.0
    
def transform_data(dataframe):
    # convert data and show the new info
    # activity
    dataframe['activity'] = dataframe['activity'].replace(['Unknown'], '1')
    dataframe['activity'] = dataframe['activity'].replace(['Still'], '2')
    dataframe['activity'] = dataframe['activity'].replace(['Tilting'], '3')
    dataframe['activity'] = dataframe['activity'].replace(['Exercise'], '4')
    dataframe['activity'] = dataframe['activity'].replace(['In vehicle'], '5')
    # class
    dataframe['class'] = dataframe['class'].replace(['awake'], '0')
    dataframe['class'] = dataframe['class'].replace(['asleep'], '1')
    # day_of_week
    dataframe['day_of_week'] = dataframe['day_of_week'].replace(['Tuesday'], '1')
    dataframe['day_of_week'] = dataframe['day_of_week'].replace(['Wednesday'], '2')
    dataframe['day_of_week'] = dataframe['day_of_week'].replace(['Thursday'], '3')
    dataframe['day_of_week'] = dataframe['day_of_week'].replace(['Friday'], '4')
    dataframe['day_of_week'] = dataframe['day_of_week'].replace(['Saturday'], '5')
    dataframe['day_of_week'] = dataframe['day_of_week'].replace(['Sunday'], '6')
    dataframe['day_of_week'] = dataframe['day_of_week'].replace(['Monday'], '7')
    # phone_lock
    dataframe['phone_lock'] = dataframe['phone_lock'].replace(['not locked'], '0')
    dataframe['phone_lock'] = dataframe['phone_lock'].replace(['locked'], '1')
    # location
    dataframe['location'] = dataframe['location'].replace(['Other'], '1')
    dataframe['location'] = dataframe['location'].replace(['University'], '2')
    dataframe['location'] = dataframe['location'].replace(['House'], '3')
    
    # timestamp
    if 'timestamp2' in dataframe.columns:
        dataframe['timestamp_text'] = dataframe['timestamp']
        dataframe['timestamp'] = dataframe['timestamp2']
        del dataframe['timestamp2']
    if 'outside_range' in dataframe.columns:
        del dataframe['outside_range']
    if 'time_sleeping' in dataframe.columns:
        del dataframe['time_sleeping']
    if 'wakeup_time' in dataframe.columns:
        del dataframe['wakeup_time']
    if 'sleep_time' in dataframe.columns:
        del dataframe['sleep_time']
    if 'distanceHome' in dataframe.columns:
        del dataframe['distanceHome']    
    if 'showActuation' in dataframe.columns:
        del dataframe['showActuation'] 
    #dataframe['timestamp'] = dataframe['timestamp'].astype('datetime64[ns]')
    #dataframe['timestamp'] = dataframe['timestamp'].astype('int64')
    #dataframe['timestamp'] = datetime.strptime(dataframe['timestamp'],'%Y-%m-%d %H:%M:%S').timestamp() * 1000
    #dataframe['timestamp'] = pd.to_datetime(dataframe['timestamp'],'%Y-%m-%d %H:%M:%S') * 1000
    #dataframe['timestamp'] = dataframe['timestamp'].dt.second()
    # data_type
    #dataframe['activity'] = dataframe['activity'].astype(np.integer) 
    #dataframe['class'] = dataframe['class'].astype(np.integer) 
    #dataframe['day_of_week'] = dataframe['day_of_week'].astype(np.integer) 
    #dataframe['phone_lock'] = dataframe['phone_lock'].astype(np.integer) 
    #dataframe['location'] = dataframe['location'].astype(np.integer) 
    dataframe['sound'] = dataframe['sound'].apply(convert_to_decibels)
    dataframe['proximity'] = dataframe['proximity'].apply(treat_data_to_0_1)
    dataframe['activity'] = dataframe['activity'].astype('int32')
    dataframe['class'] = dataframe['class'].astype('int32')
    dataframe['day_of_week'] = dataframe['day_of_week'].astype('int32')
    dataframe['phone_lock'] = dataframe['phone_lock'].astype('int32') 
    dataframe['location'] = dataframe['location'].astype('int32')
    #dataframe['minutes_day'] = dataframe['minutes_day'].astype('float32')
    dataframe['time_to_next_alarm'] = dataframe['time_to_next_alarm'].astype('float32')
    dataframe['sound'] = dataframe['sound'].astype('float32')
    dataframe['light'] = dataframe['light'].astype('float32')
    dataframe['proximity'] = dataframe['proximity'].astype('float32')
    dataframe['timestamp'] = dataframe['timestamp'].astype('float32')
    
    temp = dataframe['class'].copy()
    del dataframe['class']
    dataframe['class'] = temp
    #return dataframe
    return dataframe

In [16]:
def transform_normalize_data(df):
    result = df.copy()
    for feature_name in df.columns:
        if feature_name != "class" and feature_name != "timestamp_text":
            max_value = df[feature_name].max()
            if(feature_name == "timestamp"):
                max_value = 1924905600.0 #2030-12-31 00:00:00  
            if(feature_name == "day_of_week"):
                max_value = 7            
            if(feature_name == "sound"):
                max_value = 403 # overhal higher value            
            if(feature_name == "location"):
                max_value = 3
            if(feature_name == "activity"):
                max_value = 5
            if(feature_name == "minutes_day"):
                max_value = 1439 # 24 horas
            if(feature_name == "time_to_next_alarm"):
                max_value = 604800000.0 # 2.8 years in ms
            min_value = df[feature_name].min()
            print(feature_name+" "+str(min_value)+" "+str(max_value))
            if feature_name != "showActuation":
                result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
            if(max_value == 0 and min_value == 0):
                result[feature_name] = 0.0
    return result

In [17]:
def transform_data_type(dataset):
    # diminui o tamanho do tipo de dado (float32 é mais que o suficiente)
    dataset['activity'] = dataset['activity'].astype('int32')
    dataset['location'] = dataset['location'].astype('int32')
    dataset['timestamp'] = dataset['timestamp'].astype('float32')
    dataset['day_of_week'] = dataset['day_of_week'].astype('int32')
    dataset['light'] = dataset['light'].astype('int32')
    dataset['phone_lock'] = dataset['phone_lock'].astype('int32')
    dataset['proximity'] = dataset['proximity'].astype('int32')
    dataset['sound'] = dataset['sound'].astype('float32')
    dataset['time_to_next_alarm'] = dataset['time_to_next_alarm'].astype('float32')
    
    return dataset

In [18]:
def transform_output_to_numeric(dataframe):
    dataframe['class'] = dataframe['class'].replace(['awake'], '0')
    dataframe['class'] = dataframe['class'].replace(['asleep'], '1')
    
    dataframe['class'] = dataframe['class'].astype('int32')
    
    return dataframe

In [19]:
def transform_output_to_str(dataframe):
    dataframe['class'] = dataframe['class'].apply(str)
    #print(dataframe['class'])
    dataframe['class'] = dataframe['class'].replace(to_replace='0',value='awake')
    dataframe['class'] = dataframe['class'].replace(to_replace='1',value='asleep')
    
    return dataframe

# Files by students

In [20]:
# Dataframe of all datasets
# initialize list of lists
datasets = ['0Jf4TH9Zzse0Z1Jjh7SnTOe2MMzeSnFi7feTnkG6vgs',
            '0tdmm6rwW3KquQ73ATYYJ5JkpMtvbppJ0VzA2GExdA', 
            '2cyV53lVyUtlMj0BRwilEWtYJwUiviYoL48cZBPBq0', 
            '2J22RukYnEbKTk7t+iUVDBkorcyL5NKN6TrLe89ys', 
            #['5FLZBTVAPwdq9QezHE2sVCJIs7p+r6mCemA2gp9jATk'], #does not have the file
            '7EYF5I04EVqisUJCVNHlqn77UAuOmwL2Dahxd3cA', 
            'a9Qgj8ENWrHvl9QqlXcIPKmyGMKgbfHk9Dbqon1HQP4', 
            'ae4JJBZDycEcY8McJF+3BxyvZ1619y03BNdCxzpZTc', 
            'Ch3u5Oaz96VSrQbf0z31X6jEIbeIekkC0mwPzCdeJ1U', 
            'CH8f0yZkZL13zWuE9ks1CkVJRVrr+jsGdUXHrZ6YeA', 
            'DHO1K4jgiwZJOfQTrxvKE2vn7hkjamigroGD5IaeRc', 
            'DHPqzSqSttiba1L3BD1cptNJPjSxZ8rXxF9mY3za6WA', 
            'dQEFscjqnIlug8Tgq97JohhSQPG2DEOWJqS86wCrcY', 
            'HFvs2CohmhHte+AaCzFasjzegGzxZKPhkrX23iI6Xo', 
            'jgB9E8v3Z6PKdTRTCMAijBllA9YEMtrmHbe4qsbmJWw', 
            'JkY++R7E8myldLN3on6iQ78Ee78zCbrLuggfwGju3I', 
            'K4SLohf+TN1Ak8Dn8iE3Lme7rEMPISfppB2sXfHX8', 
            'oGaWetJJJEWHuvYdWYo826SQxfhCExVVQ2da8LE1Y7Q', 
            'pyt24oiDAHsmgWMvkFKz2fn2pwcHiXchd6KchLM', 
            'PZCf1nfvhR+6fk+7+sPNMYOgb8BAMmtQtfoRS83Suc', 
            'QUNCATForxzK0HHw46LrGOMWh0eVA8Y5XWEiUXX+cQ', 
            'rIl2UK9+bQ+tzpFdbJAdbBxEa5GbgrgC030yEaENLw', 
            'RoBW3cDOO9wWRMPO2twQff83MPc+OXn6gJ+a1DafreI', 
            'SH3kQeyd5volraxw8vOyhlowNqWBPr1IJ9URNXUL4', 
            'VVpwFNMrEglveh6MDN8lrRzTy5OwzglD4FURfM4A2is', 
            'Wa1mcNmbh66S7VS6GIzyfCFMD3SGhbtDQyFP1ywJEsw', 
            'XCKRE0BWRHxfP1kZIihgtT+jUjSp2GE8v5ZlhcIhVmA', 
            'YI5Y79K6GXqAUoGP6PNyII8WKlAoel4urDxWSVVOvBw', 
            'ypklj+8GJ15rOIH1lpKQtFJOuK+VdvyCuBPqhY3aoM', 
            'ZSsAZ0Pq+MCqFrnjsRFn5Ua09pMCVaOV9c8ZuYb7XQY']
  
# Create the pandas DataFrame
folders = pd.DataFrame(datasets, columns=['folder'])

#folders

In [21]:
pd.read_csv(outputFolders+"/w0/student_0Jf4TH9Zzse0Z1Jjh7SnTOe2MMzeSnFi7feTnkG6vgs.csv")


,activity,location,distanceHome,showActuation,timestamp,time_to_next_alarm,sound,proximity,phone_lock,light,day_of_week,minutes_day,sleep_time,wakeup_time,time_sleeping,timestamp_text,class
0,Unknown,Other,0,False,1526390445,594000000.0,0,5,not locked,37,Tuesday,860,1020,660,64808,2018-05-15 14:20:45+00:00,awake
1,Unknown,Other,0,False,1526390445,594000000.0,0,5,not locked,37,Tuesday,860,1020,660,64808,2018-05-15 14:20:45+00:00,awake
2,Still,University,0,False,1526390475,593940000.0,5517,5,not locked,35,Tuesday,861,1020,660,64808,2018-05-15 14:21:15+00:00,awake
3,Still,University,0,False,1526390505,593940000.0,5517,5,not locked,307,Tuesday,861,1020,660,64808,2018-05-15 14:21:45+00:00,awake
4,Still,University,0,False,1526390535,593880000.0,5809,5,not locked,42,Tuesday,862,1020,660,64808,2018-05-15 14:22:15+00:00,awake
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2404,Still,University,0,False,1526648089,592260000.0,2489,0,locked,0,Friday,834,300,615,18900,2018-05-18 13:54:49+00:00,awake
2405,Tilting,University,0,False,1526648254,592080000.0,2489,0,locked,0,Friday,837,300,615,18900,2018-05-18 13:57:34+00:00,awake
2406,Still,University,0,False,1526648284,592020000.0,17018,0,locked,0,Friday,838,300,615,18900,2018-05-18 13:58:04+00:00,awake
2407,Tilting,University,0,False,1526648416,591900000.0,17018,0,locked,0,Friday,840,300,615,18900,2018-05-18 14:00:16+00:00,awake


In [22]:
pd.read_csv(outputFolders+"/w0/student_0Jf4TH9Zzse0Z1Jjh7SnTOe2MMzeSnFi7feTnkG6vgs"+input_file_sufix)


,activity,location,distanceHome,showActuation,timestamp,time_to_next_alarm,sound,proximity,phone_lock,light,day_of_week,minutes_day,sleep_time,wakeup_time,time_sleeping,timestamp_text,class,outside_range
0,Unknown,Other,0,False,1526570494,589380000.0,0,5,not locked,168,Thursday,981,1335,180,17100,2018-05-17 16:21:34+00:00,awake,0
1,Unknown,University,0,False,1526570554,589320000.0,5533,5,not locked,73,Thursday,982,1335,180,17100,2018-05-17 16:22:34+00:00,awake,0
2,Still,University,0,False,1526570584,589260000.0,6348,5,not locked,123,Thursday,983,1335,180,17100,2018-05-17 16:23:04+00:00,awake,0
3,Unknown,University,0,False,1526570614,589260000.0,6348,5,not locked,75,Thursday,983,1335,180,17100,2018-05-17 16:23:34+00:00,awake,0
4,Still,University,0,False,1526570674,589200000.0,21849,5,not locked,107,Thursday,984,1335,180,17100,2018-05-17 16:24:34+00:00,awake,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1326,In vehicle,Other,0,False,1526648251,598020000.0,8747,0,locked,86,Friday,837,390,870,28800,2018-05-18 13:57:31+00:00,awake,0
1327,In vehicle,Other,0,False,1526648311,597960000.0,14110,5,not locked,292,Friday,838,390,870,28800,2018-05-18 13:58:31+00:00,awake,0
1328,Exercise,Other,0,False,1526648431,597840000.0,10074,0,locked,47,Friday,840,390,870,28800,2018-05-18 14:00:31+00:00,awake,0
1329,In vehicle,Other,0,False,1526648552,597720000.0,14001,5,not locked,152,Friday,842,390,870,28800,2018-05-18 14:02:32+00:00,awake,0


In [30]:
for weekIndex in range(0,numberOfWeeks):
    print()
    print()
    print("Starting week",weekIndex)
    print()
    print()
    for indexFolder, folder in folders.iterrows():
        fileNameX = outputFoldersWeek+str(weekIndex)+"/student_"+folder['folder']+input_file_sufix
        dataset = pd.read_csv(fileNameX)
        print()
        print(fileNameX, Counter(dataset['class']) )
        # basic transformations 
        datasetNumericClassification = transform_data(dataset)
        datasetNumericClassification = transform_data_type(datasetNumericClassification)
        datasetNumericClassificationNormalized = transform_normalize_data(datasetNumericClassification)
        datasetNumericClassificationNormalized = transform_output_to_str(datasetNumericClassificationNormalized)

        datasetNumericClassificationNormalized.to_csv(outputFoldersWeek+str(weekIndex)+"/student_"+folder['folder']+fileOutPutSufixName, sep=',', encoding='utf-8', index=False)



Starting week 0



../data_2019_processed/w0/student_0Jf4TH9Zzse0Z1Jjh7SnTOe2MMzeSnFi7feTnkG6vgs_filtred.csv Counter({'asleep': 905, 'awake': 426})
activity 1 5
location 1 3
timestamp 1526570500.0 1924905600.0
time_to_next_alarm 59997.0 604800000.0
sound 0.0 403
proximity 0 1
phone_lock 0 1
light 0 7314
day_of_week 3 7
minutes_day 0 1439

../data_2019_processed/w0/student_0tdmm6rwW3KquQ73ATYYJ5JkpMtvbppJ0VzA2GExdA_filtred.csv Counter({'asleep': 905, 'awake': 426})
activity 1 5
location 1 3
timestamp 1526570500.0 1924905600.0
time_to_next_alarm 59997.0 604800000.0
sound 0.0 403
proximity 0 1
phone_lock 0 1
light 0 7314
day_of_week 3 7
minutes_day 0 1439

../data_2019_processed/w0/student_2cyV53lVyUtlMj0BRwilEWtYJwUiviYoL48cZBPBq0_filtred.csv Counter({'asleep': 905, 'awake': 426})
activity 1 5
location 1 3
timestamp 1526570500.0 1924905600.0
time_to_next_alarm 59997.0 604800000.0
sound 0.0 403
proximity 0 1
phone_lock 0 1
light 0 7314
day_of_week 3 7
minutes_day 0 1439

../data_2019_pr

activity 1 5
location 1 3
timestamp 1526648700.0 1924905600.0
time_to_next_alarm 0.0 604800000.0
sound 0.0 403
proximity 0 1
phone_lock 0 1
light 0 29794
day_of_week 1 7
minutes_day 0 1439

../data_2019_processed/w1/student_0tdmm6rwW3KquQ73ATYYJ5JkpMtvbppJ0VzA2GExdA_filtred.csv Counter({'awake': 4045, 'asleep': 2142})
activity 1 5
location 1 3
timestamp 1526648700.0 1924905600.0
time_to_next_alarm 0.0 604800000.0
sound 0.0 403
proximity 0 1
phone_lock 0 1
light 0 29794
day_of_week 1 7
minutes_day 0 1439

../data_2019_processed/w1/student_2cyV53lVyUtlMj0BRwilEWtYJwUiviYoL48cZBPBq0_filtred.csv Counter({'awake': 4045, 'asleep': 2142})
activity 1 5
location 1 3
timestamp 1526648700.0 1924905600.0
time_to_next_alarm 0.0 604800000.0
sound 0.0 403
proximity 0 1
phone_lock 0 1
light 0 29794
day_of_week 1 7
minutes_day 0 1439

../data_2019_processed/w1/student_2J22RukYnEbKTk7t+iUVDBkorcyL5NKN6TrLe89ys_filtred.csv Counter({'awake': 4045, 'asleep': 2142})
activity 1 5
location 1 3
timestamp 15266


../data_2019_processed/w1/student_ZSsAZ0Pq+MCqFrnjsRFn5Ua09pMCVaOV9c8ZuYb7XQY_filtred.csv Counter({'awake': 4045, 'asleep': 2142})
activity 1 5
location 1 3
timestamp 1526648700.0 1924905600.0
time_to_next_alarm 0.0 604800000.0
sound 0.0 403
proximity 0 1
phone_lock 0 1
light 0 29794
day_of_week 1 7
minutes_day 0 1439


Starting week 2



../data_2019_processed/w2/student_0Jf4TH9Zzse0Z1Jjh7SnTOe2MMzeSnFi7feTnkG6vgs_filtred.csv Counter({'awake': 3769, 'asleep': 1455})
activity 1 5
location 1 3
timestamp 1527350900.0 1924905600.0
time_to_next_alarm 0.0 604800000.0
sound 0.0 403
proximity 0 1
phone_lock 0 1
light 0 33895
day_of_week 1 7
minutes_day 0 1439

../data_2019_processed/w2/student_0tdmm6rwW3KquQ73ATYYJ5JkpMtvbppJ0VzA2GExdA_filtred.csv Counter({'awake': 3769, 'asleep': 1455})
activity 1 5
location 1 3
timestamp 1527350900.0 1924905600.0
time_to_next_alarm 0.0 604800000.0
sound 0.0 403
proximity 0 1
phone_lock 0 1
light 0 33895
day_of_week 1 7
minutes_day 0 1439

../data_2019_proc


../data_2019_processed/w2/student_XCKRE0BWRHxfP1kZIihgtT+jUjSp2GE8v5ZlhcIhVmA_filtred.csv Counter({'awake': 3769, 'asleep': 1455})
activity 1 5
location 1 3
timestamp 1527350900.0 1924905600.0
time_to_next_alarm 0.0 604800000.0
sound 0.0 403
proximity 0 1
phone_lock 0 1
light 0 33895
day_of_week 1 7
minutes_day 0 1439

../data_2019_processed/w2/student_YI5Y79K6GXqAUoGP6PNyII8WKlAoel4urDxWSVVOvBw_filtred.csv Counter({'awake': 3769, 'asleep': 1455})
activity 1 5
location 1 3
timestamp 1527350900.0 1924905600.0
time_to_next_alarm 0.0 604800000.0
sound 0.0 403
proximity 0 1
phone_lock 0 1
light 0 33895
day_of_week 1 7
minutes_day 0 1439

../data_2019_processed/w2/student_ypklj+8GJ15rOIH1lpKQtFJOuK+VdvyCuBPqhY3aoM_filtred.csv Counter({'awake': 3769, 'asleep': 1455})
activity 1 5
location 1 3
timestamp 1527350900.0 1924905600.0
time_to_next_alarm 0.0 604800000.0
sound 0.0 403
proximity 0 1
phone_lock 0 1
light 0 33895
day_of_week 1 7
minutes_day 0 1439

../data_2019_processed/w2/student_ZSs


../data_2019_processed/w3/student_VVpwFNMrEglveh6MDN8lrRzTy5OwzglD4FURfM4A2is_filtred.csv Counter({'awake': 646, 'asleep': 393})
activity 1 5
location 1 3
timestamp 1527945100.0 1924905600.0
time_to_next_alarm 0.0 604800000.0
sound 0.0 403
proximity 0 1
phone_lock 0 1
light 0 2796
day_of_week 5 7
minutes_day 0 1439

../data_2019_processed/w3/student_Wa1mcNmbh66S7VS6GIzyfCFMD3SGhbtDQyFP1ywJEsw_filtred.csv Counter({'awake': 646, 'asleep': 393})
activity 1 5
location 1 3
timestamp 1527945100.0 1924905600.0
time_to_next_alarm 0.0 604800000.0
sound 0.0 403
proximity 0 1
phone_lock 0 1
light 0 2796
day_of_week 5 7
minutes_day 0 1439

../data_2019_processed/w3/student_XCKRE0BWRHxfP1kZIihgtT+jUjSp2GE8v5ZlhcIhVmA_filtred.csv Counter({'awake': 646, 'asleep': 393})
activity 1 5
location 1 3
timestamp 1527945100.0 1924905600.0
time_to_next_alarm 0.0 604800000.0
sound 0.0 403
proximity 0 1
phone_lock 0 1
light 0 2796
day_of_week 5 7
minutes_day 0 1439

../data_2019_processed/w3/student_YI5Y79K6GXq

In [31]:
datasetNumericClassificationNormalized

,activity,location,timestamp,time_to_next_alarm,sound,proximity,phone_lock,light,day_of_week,minutes_day,timestamp_text,class
0,0.00,0.0,0.000000e+00,0.987401,0.000000,1.0,0.0,0.206366,0.0,0.591383,2018-06-02 14:11:34+00:00,awake
1,0.25,0.5,0.000000e+00,0.987302,0.629973,1.0,0.0,0.497139,0.0,0.592078,2018-06-02 14:12:04+00:00,awake
2,0.00,0.5,3.224502e-07,0.987302,0.629973,1.0,0.0,0.079399,0.0,0.592078,2018-06-02 14:12:34+00:00,awake
3,0.00,0.5,3.224502e-07,0.987202,0.569047,1.0,0.0,0.145207,0.0,0.592773,2018-06-02 14:13:04+00:00,awake
4,0.00,0.5,3.224502e-07,0.987202,0.569047,1.0,0.0,0.151288,0.0,0.592773,2018-06-02 14:13:34+00:00,awake
...,...,...,...,...,...,...,...,...,...,...,...,...
1034,1.00,0.0,5.320428e-04,0.923909,0.577338,1.0,0.0,0.003219,1.0,0.035441,2018-06-05 00:51:21+00:00,awake
1035,1.00,0.0,5.320428e-04,0.923810,0.612109,1.0,0.0,0.002861,1.0,0.036136,2018-06-05 00:52:21+00:00,awake
1036,0.00,0.0,5.323653e-04,0.923611,0.573415,1.0,0.0,0.001431,1.0,0.037526,2018-06-05 00:54:11+00:00,awake
1037,0.75,0.0,5.326878e-04,0.923611,0.538685,1.0,0.0,0.001073,1.0,0.037526,2018-06-05 00:54:41+00:00,awake


In [32]:
print("Test")
pd.read_csv(outputFolders+"/w0/student_YI5Y79K6GXqAUoGP6PNyII8WKlAoel4urDxWSVVOvBw"+fileOutPutSufixName)

Test


,activity,location,timestamp,time_to_next_alarm,sound,proximity,phone_lock,light,day_of_week,minutes_day,timestamp_text,class
0,0.00,0.0,0.000000e+00,0.974501,0.000000,1.0,0.0,0.022970,0.00,0.681723,2018-05-17 16:21:34+00:00,awake
1,0.00,0.5,0.000000e+00,0.974402,0.604264,1.0,0.0,0.009981,0.00,0.682418,2018-05-17 16:22:34+00:00,awake
2,0.25,0.5,3.213375e-07,0.974303,0.597445,1.0,0.0,0.016817,0.00,0.683113,2018-05-17 16:23:04+00:00,awake
3,0.00,0.5,3.213375e-07,0.974303,0.597445,1.0,0.0,0.010254,0.00,0.683113,2018-05-17 16:23:34+00:00,awake
4,0.25,0.5,3.213375e-07,0.974204,0.536104,1.0,0.0,0.014629,0.00,0.683808,2018-05-17 16:24:34+00:00,awake
...,...,...,...,...,...,...,...,...,...,...,...,...
1326,1.00,0.0,1.950519e-04,0.988789,0.581536,0.0,1.0,0.011758,0.25,0.581654,2018-05-18 13:57:31+00:00,awake
1327,1.00,0.0,1.953732e-04,0.988689,0.557805,1.0,0.0,0.039923,0.25,0.582349,2018-05-18 13:58:31+00:00,awake
1328,0.75,0.0,1.956945e-04,0.988491,0.574526,0.0,1.0,0.006426,0.25,0.583739,2018-05-18 14:00:31+00:00,awake
1329,1.00,0.0,1.960159e-04,0.988293,0.558190,1.0,0.0,0.020782,0.25,0.585129,2018-05-18 14:02:32+00:00,awake
